In [9]:
import numpy as np

# Import the functions from the previous snippet
# (here I just redefine them quickly)
def S_from_cmax(cmax, L, d, W, Wg, D, C):
    k = np.sqrt(C/D)
    z = k*L
    return (2*D/L**2) * cmax * (1.0 + (k/np.tanh(z)) * (L + (W/Wg)*d))

def dS_partials(cmax, L, d, W, Wg, D, C):
    G = L + (W/Wg)*d
    k = np.sqrt(C/D)
    z = k*L
    coth = 1.0/np.tanh(z)
    csch2 = 1.0/np.sinh(z)**2

    dS_dcmax = (2*D/L**2) * (1.0 + G*k*coth)
    dS_dD = (2.0/L**2) * cmax * (
        1.0 + G*k*coth
        - 0.5*G*k*(coth - k*L*csch2)
    )
    dS_dC = (2.0/L**2) * cmax * ( G/(2.0*k) ) * (coth - k*L*csch2)

    return dS_dcmax, dS_dD, dS_dC

def sigma_S(cmax, L, d, W, Wg, D, C,
            sigma_cmax, sigma_D, sigma_C,
            rho_cmaxD=0.0, rho_cmaxC=0.0, rho_DC=0.0):
    dR, dD, dC = dS_partials(cmax, L, d, W, Wg, D, C)

    var = (dR**2)*sigma_cmax**2 + (dD**2)*sigma_D**2 + (dC**2)*sigma_C**2
    var += 2.0*dR*dD * rho_cmaxD * sigma_cmax * sigma_D
    var += 2.0*dR*dC * rho_cmaxC * sigma_cmax * sigma_C
    var += 2.0*dD*dC * rho_DC     * sigma_D     * sigma_C

    return np.sqrt(max(var, 0.0))

# -------------------------------
# Example parameters
# -------------------------------
L     = 50.0     # µm
d     = 3.0      # µm
W     = 50.0     # µm
Wg    = 0.25*W   # µm

cmax  = 0.02     # interface concentration
D     = 10.0     # µm^2/s
C     = 0.10     # 1/s

# Uncertainties (1-sigma)
sigma_cmax = 0.002   # (10% relative)
sigma_D    = 1.0     # (10% relative)
sigma_C    = 0.02    # (20% relative)

# -------------------------------
# Compute S and its propagated error
# -------------------------------
S_mean = S_from_cmax(cmax, L, d, W, Wg, D, C)
S_err  = sigma_S(cmax, L, d, W, Wg, D, C,
                 sigma_cmax, sigma_D, sigma_C)

print(f"Production rate S = {S_mean:.5f} ± {S_err:.5f}  [1/s]")
print(f"(relative uncertainty ~ {100*S_err/S_mean:.1f}%)")


Production rate S = 0.00115 ± 0.00017  [1/s]
(relative uncertainty ~ 14.4%)


In [20]:

import numpy as np
import sympy as sp
import matplotlib.pyplot as plt
from itertools import product

# PNAS figure style
final_width_cm = 10    # width in cm
final_height_cm = 5   # height in cm
final_width_in = final_width_cm / 2.54
final_height_in = final_height_cm / 2.54
pnas_fontsize = 8  # points
plt.rcParams.update({
    'font.size': pnas_fontsize,
    'pdf.fonttype': 42,
    'ps.fonttype': 42
})

# --- Geometry (shared) ---
L_val = 50.0                # µm Length of producer & consumer regions (each)
d_val = 3.0                 # µm Length of diffusive gap
W_val = 50.0                # µm Width of producer/consumer chambers
W_gap_val = 0.5/2*W_val     # µm Width of the diffusive gap (== 0.25 * W_val)

# === HQNO (Species A) - IN ng/mL ===
# Uncertainty method: median - Q25
cmax_med_A = 92.186383      # ng/mL - Median maximum concentration
cmax_q25_A = 69.251959      # ng/mL - 25th percentile
sigma_cmax_A = cmax_med_A - cmax_q25_A  # ng/mL - Uncertainty = 22.934424 ng/mL

D_A_mid = 40.0              # µm²/s - Median diffusion coefficient
sigma_D_A = 4.0             # µm²/s - Uncertainty in D
D_A_rng = (D_A_mid - sigma_D_A, D_A_mid, D_A_mid + sigma_D_A)  # (36.0, 40.0, 44.0)

S_A_mid = 23.97             # ng/(mL·s) - Median production rate (converted from µg to ng)
sigma_S_A = 0.74            # ng/(mL·s) - Uncertainty in S (converted from µg to ng)
S_A_rng = (S_A_mid - sigma_S_A, S_A_mid, S_A_mid + sigma_S_A)  # (23.23, 23.97, 24.71)

C_A_mid = 0.52836           # s⁻¹ - Median uptake rate
sigma_C_A = 0.33099         # s⁻¹ - Uncertainty in C
C_A_rng = (C_A_mid - sigma_C_A, C_A_mid, C_A_mid + sigma_C_A)  # (0.19736, 0.52836, 0.85936)

# === RHL (Species B) - FILL IN YOUR VALUES HERE ===
# TODO: Replace these placeholder values with your actual RHL data
cmax_med_B = 100.0          # ng/mL - PLACEHOLDER: Median maximum concentration
cmax_q25_B = 80.0           # ng/mL - PLACEHOLDER: 25th percentile
sigma_cmax_B = cmax_med_B - cmax_q25_B  # ng/mL - Uncertainty

D_B_mid = 15.0              # µm²/s - PLACEHOLDER: Median diffusion coefficient
sigma_D_B = 5.0             # µm²/s - PLACEHOLDER: Uncertainty in D
D_B_rng = (D_B_mid - sigma_D_B, D_B_mid, D_B_mid + sigma_D_B)  # (10.0, 15.0, 20.0)

S_B_mid = 4.60031           # µg/(mL·s) - PLACEHOLDER: Median production rate
sigma_S_B = 1.49832         # µg/(mL·s) - PLACEHOLDER: Uncertainty in S
S_B_rng = (S_B_mid - sigma_S_B, S_B_mid, S_B_mid + sigma_S_B)  # (3.10199, 4.60031, 6.09863)

C_B_mid = 0.11585           # s⁻¹ - PLACEHOLDER: Median uptake rate
sigma_C_B = 0.07029         # s⁻¹ - PLACEHOLDER: Uncertainty in C
C_B_rng = (C_B_mid - sigma_C_B, C_B_mid, C_B_mid + sigma_C_B)  # (0.04556, 0.11585, 0.18614)

# --- Symbolic setup (solve once) ---
L, d, D, S, C, W, W_gap, x = sp.symbols('L d D S C W W_gap x', positive=True)
k = sp.sqrt(C / D)

A_sym, C1_sym, C2_sym, M_sym = sp.symbols('A C1 C2 M')

# Piecewise solution forms:
R_I   = -S/(2*D)*x**2 + A_sym*x                         # Producer with source S
R_II  = C1_sym*x + C2_sym                               # Gap (pure diffusion)
R_III = M_sym*sp.sinh(k*(2*L + d - x))                  # Consumer with sink C

# Derivatives
R_Ip, R_IIp, R_IIIp = sp.diff(R_I, x), sp.diff(R_II, x), sp.diff(R_III, x)

# BCs + width-weighted flux continuity
eqs = [
    sp.Eq(R_I.subs(x, 0), 0),                                # R(0) = 0
    sp.Eq(R_III.subs(x, 2*L + d), 0),                        # R(2L+d) = 0
    sp.Eq(R_I.subs(x, L), R_II.subs(x, L)),                  # Continuity at x=L
    sp.Eq(W * R_Ip.subs(x, L), W_gap * R_IIp.subs(x, L)),    # Flux continuity at x=L
    sp.Eq(R_II.subs(x, L + d), R_III.subs(x, L + d)),        # Continuity at x=L+d
    sp.Eq(W_gap * R_IIp.subs(x, L + d), W * R_IIIp.subs(x, L + d)) # Flux continuity at x=L+d
]

solution_expr = sp.solve(eqs, (A_sym, C1_sym, C2_sym, M_sym), dict=True)[0]

def conc_curve(D_val, S_val, C_val, x_vals):
    """Vectorized steady-state concentration R(x) for given D,S,C."""
    num = {L: L_val, d: d_val, W: W_val, W_gap: W_gap_val, D: D_val, S: S_val, C: C_val}
    A_val  = float(solution_expr[A_sym].subs(num))
    C1_val = float(solution_expr[C1_sym].subs(num))
    C2_val = float(solution_expr[C2_sym].subs(num))
    M_val  = float(solution_expr[M_sym].subs(num))
    k_val  = float(sp.sqrt(C_val / D_val))

    x_arr = np.asarray(x_vals, dtype=float)
    out = np.empty_like(x_arr)

    m1 = x_arr < L_val
    m2 = (x_arr >= L_val) & (x_arr < L_val + d_val)
    m3 = ~(m1 | m2)

    out[m1] = -S_val/(2 * D_val) * x_arr[m1]**2 + A_val * x_arr[m1]
    out[m2] = C1_val * x_arr[m2] + C2_val
    out[m3] = M_val * np.sinh(k_val * (2 * L_val + d_val - x_arr[m3]))
    return out

def envelope_for_ranges(D_rng, S_rng, C_rng, x_vals):
    """Mean curve = mid parameters. Envelope = pointwise min/max over corners."""
    D_lo, D_mid, D_hi = D_rng
    S_lo, S_mid, S_hi = S_rng
    C_lo, C_mid, C_hi = C_rng

    R_mean = conc_curve(D_mid, S_mid, C_mid, x_vals)

    corners = list(product([D_lo, D_hi], [S_lo, S_hi], [C_lo, C_hi]))
    R_min = np.full_like(x_vals, np.inf, dtype=float)
    R_max = np.full_like(x_vals, -np.inf, dtype=float)
    for (Dv, Sv, Cv) in corners:
        R = conc_curve(Dv, Sv, Cv, x_vals)
        R_min = np.minimum(R_min, R)
        R_max = np.maximum(R_max, R)
    return R_mean, R_min, R_max

# Build curves
x_vals = np.linspace(0, 2*L_val + d_val, 600)
R_A_mean, R_A_min, R_A_max = envelope_for_ranges(D_A_rng, S_A_rng, C_A_rng, x_vals)
R_B_mean, R_B_min, R_B_max = envelope_for_ranges(D_B_rng, S_B_rng, C_B_rng, x_vals)

# Print statistics
print("="*70)
print("PARAMETER SUMMARY")
print("="*70)
print(f"Species A (HQNO):")
print(f"  Concentration: {cmax_med_A:.2f} ± {sigma_cmax_A:.2f} ng/mL")
print(f"  D = {D_A_mid:.1f} ± {sigma_D_A:.1f} µm²/s (range: {D_A_rng[0]:.1f}–{D_A_rng[2]:.1f})")
print(f"  S = {S_A_mid:.2f} ± {sigma_S_A:.2f} ng/(mL·s) (range: {S_A_rng[0]:.2f}–{S_A_rng[2]:.2f})")
print(f"  C = {C_A_mid:.5f} ± {sigma_C_A:.5f} s⁻¹ (range: {C_A_rng[0]:.5f}–{C_A_rng[2]:.5f})")
print(f"  Mean concentration: {R_A_mean.mean():.2f} ng/mL")
print()
print(f"Species B (RHL):")
print(f"  Concentration: {cmax_med_B:.2f} ± {sigma_cmax_B:.2f} ng/mL (PLACEHOLDER)")
print(f"  D = {D_B_mid:.1f} ± {sigma_D_B:.1f} µm²/s (range: {D_B_rng[0]:.1f}–{D_B_rng[2]:.1f})")
print(f"  S = {S_B_mid:.5f} ± {sigma_S_B:.5f} µg/(mL·s) (range: {S_B_rng[0]:.5f}–{S_B_rng[2]:.5f})")
print(f"  C = {C_B_mid:.5f} ± {sigma_C_B:.5f} s⁻¹ (range: {C_B_rng[0]:.5f}–{C_B_rng[2]:.5f})")
print(f"  Mean concentration: {R_B_mean.mean():.4f} µg/mL")
print("="*70)

PARAMETER SUMMARY
Species A (HQNO):
  Concentration: 92.19 ± 22.93 ng/mL
  D = 40.0 ± 4.0 µm²/s (range: 36.0–44.0)
  S = 23.97 ± 0.74 ng/(mL·s) (range: 23.23–24.71)
  C = 0.52836 ± 0.33099 s⁻¹ (range: 0.19737–0.85935)
  Mean concentration: 125.90 ng/mL

Species B (RHL):
  Concentration: 100.00 ± 20.00 ng/mL (PLACEHOLDER)
  D = 15.0 ± 5.0 µm²/s (range: 10.0–20.0)
  S = 4.60031 ± 1.49832 µg/(mL·s) (range: 3.10199–6.09863)
  C = 0.11585 ± 0.07029 s⁻¹ (range: 0.04556–0.18614)
  Mean concentration: 69.5919 µg/mL
